In [ ]:
from config import api_key
import requests
import time as t
import pandas as pd
import matplotlib.pyplot as plt
from citipy import citipy as c
import random as r

date = t.strftime("%d %b %Y", t.gmtime())
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

base_url = f"{url}appid={api_key}&units={units}&q="

In [ ]:
cities = []
countries = []
for i in range(2500):
    lat = r.uniform(-90, 90)
    lng = r.uniform(-180, 180)
    a = c.nearest_city(lat, lng)
    cities.append(a.city_name)
    countries.append(a.country_code)

cities_df = pd.DataFrame({"city": cities, "country": countries,
                          "latitude": "", "temp": "", "humidity": "",
                          "clouds": "", "winds": ""})
cities_df.drop_duplicates(subset='city', inplace=True)
print(cities_df['city'].count())
cities_df.head()

In [ ]:
for i, row in cities_df.iterrows():
    query_url = f"{base_url}{row['city']},{row['country']}"
    
    print(f"Retrieving data for city {i}...{row['city']}, {row['country']}")
    
    x = t.time()

    request = requests.get(query_url).json()
    
    try:
        cities_df.loc[i, 'latitude'] = request['coord']['lat'] # latitude
        cities_df.loc[i, 'temp'] = request['main']['temp_max'] # temperature F
        cities_df.loc[i, 'humidity'] = request['main']['humidity'] # humidity %
        cities_df.loc[i, 'clouds'] = request['clouds']['all'] # cloudiness %
        cities_df.loc[i, 'winds'] = request['wind']['speed'] # wind speed kts
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        cities_df.drop(i, inplace=True)
          
    while t.time() < (x + 1):
          t.sleep(.1)
          
cities_df.reset_index(drop=True, inplace=True)

In [ ]:
cities_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

xmin = int((cities_df['temp'].min() // 5) * 5)
xmax = int((cities_df['temp'].max() // 5) * 5 + 5)
xrange = range(xmin, xmax + 5, 5)

ax.scatter(cities_df['temp'], cities_df['latitude'], c='g', alpha=0.5, label='City')
ax.plot(xrange, [0] * len(xrange), 'r-', label='Equator')
ax.plot(xrange, [23.43678] * len(xrange), 'b:', label='Tropics')
ax.plot(xrange, [-23.43678] * len(xrange), 'b:')

plt.xticks(xrange)
plt.xlim(xrange[0], xrange[-1])
plt.legend()
plt.grid(True)
plt.title(f"City Latitude vs Max Temperature ({date})", fontsize=18)
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Max Temperature (F)", fontsize=14)
plt.savefig(f"Output/TemperatureInWorldCities_{date}.png")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

xmin = int((cities_df['humidity'].min() // 5) * 5)
xmax = int((cities_df['humidity'].max() // 5) * 5 + 5)
xrange = range(xmin, xmax + 5, 5)

ax.scatter(cities_df['humidity'], cities_df['latitude'], c='g', alpha=0.5, label='City')
ax.plot(xrange, [0] * len(xrange), 'r-', label='Equator')
ax.plot(xrange, [23.43678] * len(xrange), 'b:', label='Tropics')
ax.plot(xrange, [-23.43678] * len(xrange), 'b:')

plt.xticks(xrange)
plt.xlim(xrange[0], xrange[-1])
plt.legend()
plt.grid(True)
plt.title(f"City Latitude vs Humidity ({date})", fontsize=18)
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Humidity (%)", fontsize=14)
plt.savefig(f"Output/HumidityInWorldCities_{date}.png")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

xmin = int((cities_df['clouds'].min() // 5) * 5)
xmax = int((cities_df['clouds'].max() // 5) * 5 + 5)
xrange = range(xmin, xmax + 5, 5)

ax.scatter(cities_df['clouds'], cities_df['latitude'], c='g', alpha=0.5, label='City')
ax.plot(xrange, [0] * len(xrange), 'r-', label='Equator')
ax.plot(xrange, [23.43678] * len(xrange), 'b:', label='Tropics')
ax.plot(xrange, [-23.43678] * len(xrange), 'b:')

plt.xticks(xrange)
plt.xlim(xrange[0], xrange[-1])
plt.legend()
plt.grid(True)
plt.title(f"City Latitude vs Cloud Coverage ({date})", fontsize=18)
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Cloud Coverage (%)", fontsize=14)
plt.savefig(f"Output/CloudsInWorldCities_{date}.png")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

xmin = int((cities_df['winds'].min() // 5) * 5)
xmax = int((cities_df['winds'].max() // 5) * 5 + 5)
xrange = range(xmin, xmax + 5, 5)

ax.scatter(cities_df['winds'], cities_df['latitude'], c='g', alpha=0.5, label='City')
ax.plot(xrange, [0] * len(xrange), 'r-', label='Equator')
ax.plot(xrange, [23.43678] * len(xrange), 'b:', label='Tropics')
ax.plot(xrange, [-23.43678] * len(xrange), 'b:')

plt.xticks(xrange)
plt.xlim(xrange[0], xrange[-1])
plt.legend()
plt.grid(True)
plt.title(f"City Latitude vs Wind Speed ({date})", fontsize=18)
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Wind Speed (kts)", fontsize=14)
plt.savefig(f"Output/WindsInWorldCities_{date}.png")
plt.show()

In [ ]:
cities_df.to_csv(path_or_buf='Output/Cities.csv', index=False)